# 整理SLE_GSE255441

[](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE255441)

(未见刊)

肠道微生物群失调在系统性红斑狼疮（SLE）发病机制中的作用仍然难以捉摸。在这里，我们发现从健康小鼠到狼疮小鼠的粪便微生物群移植（FMT）可以改善狼疮样症状。微生物群重建有效地减少了系统类别转换重组，并提高了IGH幼稚同种型。微生物群分析显示，FMT后约氏乳杆菌富集，与嘌呤代谢产物显著相关。重要的是，约翰氏乳杆菌衍生的肌苷是嘌呤代谢的中间代谢产物，通过阻碍B细胞分化和减少肾脏B细胞浸润，有效缓解了狼疮样症状。我们进一步证明肌苷通过ERK-HIF-1α信号通路重新编程B细胞。



kidney samples from pristane-induced C57BL/6j lupus mice treated with saline (n=2) and fecal microbiota transplanation (FMT) from MRL/MpJ mice (n=3).



小鼠 肾脏 SLE小鼠2只 (移植了健康小鼠粪便的)SLE小鼠3只

这数据真好啊，简直完美符合需求

Public on Mar 31, 2024。。。。可恶，注释也没给啊啊啊啊

|||||
|:-|:-|:-|:-|
|GSM8072526|15|replicate 1|scRNA-seq|
|GSM8072527|D2|replicate 2|scRNA-seq|
|GSM8072528|B42|replicate 1|scRNA-seq|
|GSM8072529|B48|replicate 2|scRNA-seq|
|GSM8072530|B50|replicate 3|scRNA-seq|


这么推测，B42,B48,B50是FMT的小鼠

~使用标签转移~别偷懒，改成手动注释，后面可能还要注释亚群

In [1]:
import sys
from pathlib import Path
p_root = Path("~licanchengup/link/res_publish").expanduser()
None if str(p_root) in sys.path else sys.path.append(str(p_root))

In [2]:
from  func import *

/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-15 00:30:21.527477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



-------------------------help-------------------------
> parameter
    p_root	[name] res_publish
        p_run, p_plot, p_res, p_cache, p_pdf
    p_df_varmap
    map_sp_reverse
    rng
> run
    run_cross_species_models
    h5ad_to_mtx
    load_adata
    get_path_varmap
    find_path_from_para
    load_normalized_adata

> res
    get_test_result_df
    get_res_obs
    get_adata_umap
    show_umap

> plot
    get_color_map
    show_color_map
    show_color
    plot_umap
    savefig



In [3]:
df_path = pd.DataFrame({
        'path':Path('GSE255441_RAW').iterdir()
})
df_path = df_path[df_path['path'].apply(lambda x:x.match('*gz'))]
df_path['name'] = df_path['path'].apply(lambda x:x.name)
df_path = df_path.join(
    df_path['name'].str.extract(
        "^(?P<gsm_id>[^_]+)_(?P<sample>[^_]+)_(?P<type>[^_]+)\.\w{3}\.gz")
)
df_path = df_path.pivot(index = 'sample',columns='type',values='path').reset_index()
# assert df_path.index.is_unique,'[Error] not uniuqe'
df_path['treatment'] = 'SLE'
df_path['treatment'] = df_path['treatment'].mask(
    df_path['sample'].str.startswith('B'),
    'SLE_FMT')
df_path

type,sample,barcodes,features,matrix,treatment
0,B42,GSE255441_RAW/GSM8072528_B42_barcodes.tsv.gz,GSE255441_RAW/GSM8072528_B42_features.tsv.gz,GSE255441_RAW/GSM8072528_B42_matrix.mtx.gz,SLE_FMT
1,B48,GSE255441_RAW/GSM8072529_B48_barcodes.tsv.gz,GSE255441_RAW/GSM8072529_B48_features.tsv.gz,GSE255441_RAW/GSM8072529_B48_matrix.mtx.gz,SLE_FMT
2,B50,GSE255441_RAW/GSM8072530_B50_barcodes.tsv.gz,GSE255441_RAW/GSM8072530_B50_features.tsv.gz,GSE255441_RAW/GSM8072530_B50_matrix.mtx.gz,SLE_FMT
3,C15,GSE255441_RAW/GSM8072526_C15_barcodes.tsv.gz,GSE255441_RAW/GSM8072526_C15_features.tsv.gz,GSE255441_RAW/GSM8072526_C15_matrix.mtx.gz,SLE
4,D2,GSE255441_RAW/GSM8072527_D2_barcodes.tsv.gz,GSE255441_RAW/GSM8072527_D2_features.tsv.gz,GSE255441_RAW/GSM8072527_D2_matrix.mtx.gz,SLE


In [4]:
adata = {}

for i,row in df_path.iterrows():
    print('\r[{}/{}] {}'.format(i+1,df_path.shape[0],row['sample']).ljust(75,'-'))
    _adata = None
    _adata = sc.read_mtx(row['matrix']).T
    
    df_obs = pd.read_csv(row['barcodes'],header=None,index_col=0,)
    df_obs.index.name = ''
    df_var= pd.read_csv(row['features'],header=None,index_col=1,sep='\t')
    df_var.index.name = ''
    df_var.columns = 'gene_id,gene_type'.split(',')
    _adata.obs = df_obs
    _adata.var = df_var
    # display(_adata.obs.index.is_unique,_adata.var.index.is_unique)
    _adata.obs_names_make_unique()
    _adata.var_names_make_unique()
    
    _adata.obs['sample'] = row['sample']
    _adata.obs['treatment'] = row['treatment']
    # display(_adata,
    #     _adata.obs.head(2),_adata.obs.shape,
    #     _adata.var.head(2),_adata.var.shape
    # )
    
    adata[row['sample']] = _adata
    del _adata
    
print("\n[finish]\n".center(100,"-"))

[1/5] B42-----------------------------------------------------------------
[2/5] B48-----------------------------------------------------------------
[3/5] B50-----------------------------------------------------------------
[4/5] C15-----------------------------------------------------------------
[5/5] D2------------------------------------------------------------------
---------------------------------------------
[finish]
---------------------------------------------


In [5]:
adata = sc.concat(adata,index_unique=';')
adata.obs['_batch'] = df_apply_merge_field(adata.obs,'{treatment}_{sample}')
adata.obs = adata.obs.loc[:,'_batch,treatment,sample'.split(',')]
display(adata,
    adata.obs.head(2),adata.obs.shape,
    adata.var.head(2),adata.var.shape,
    adata.obs['_batch'].value_counts()
)

AnnData object with n_obs × n_vars = 26237 × 27998
    obs: '_batch', 'treatment', 'sample'

,_batch,treatment,sample
,,,
AAACCCAGTAGACAGC-1;B42,SLE_FMT_B42,SLE_FMT,B42
AAACCCAGTCTCGGAC-1;B42,SLE_FMT_B42,SLE_FMT,B42


(26237, 3)

""
""
Xkr4
Gm1992


(27998, 0)

_batch
SLE_FMT_B42    7003
SLE_C15        5877
SLE_FMT_B48    5593
SLE_D2         4338
SLE_FMT_B50    3426
Name: count, dtype: int64

In [6]:
_temp = adata.X
display(_temp[_temp > 0][:10])
del _temp

matrix([[  1.,   1.,   5., ...,  85.,  18., 233.]], dtype=float32)

## 标签转移

In [7]:
df_obs = pd.read_csv('obs_que_SLE_GSE255441.csv',index_col=0).rename(
    columns={'X_batch':'_batch'})
display(df_obs.head(2))
adata.obs = adata.obs.loc[:,[]].join(df_obs)
display(adata,
    adata.obs.head(2),adata.obs.shape,
    adata.var.head(2),adata.var.shape,
    adata.obs['_batch'].value_counts()
)

,_batch,treatment,sample,cell_type,predicted..,refUMAP_1,refUMAP_2
cell_name,,,,,,,
AAACCCAGTAGACAGC-1;B42,SLE_FMT_B42,SLE_FMT,B42,LOH,LOH,-0.791163,6.354379
AAACCCAGTCTCGGAC-1;B42,SLE_FMT_B42,SLE_FMT,B42,LOH,LOH,0.662887,4.750667


AnnData object with n_obs × n_vars = 26237 × 27998
    obs: '_batch', 'treatment', 'sample', 'cell_type', 'predicted..', 'refUMAP_1', 'refUMAP_2'

,_batch,treatment,sample,cell_type,predicted..,refUMAP_1,refUMAP_2
,,,,,,,
AAACCCAGTAGACAGC-1;B42,SLE_FMT_B42,SLE_FMT,B42,LOH,LOH,-0.791163,6.354379
AAACCCAGTCTCGGAC-1;B42,SLE_FMT_B42,SLE_FMT,B42,LOH,LOH,0.662887,4.750667


(26237, 7)

""
""
Xkr4
Gm1992


(27998, 0)

_batch
SLE_FMT_B42    7003
SLE_C15        5877
SLE_FMT_B48    5593
SLE_D2         4338
SLE_FMT_B50    3426
Name: count, dtype: int64

In [8]:
h5ad_to_mtx(adata,p_cache.joinpath('disease','SLE_mouse_LN_kidney'))

frist 10 adata.X nonzero elements:
 [[ 1  1  5  1  5 11  1  1  3 14]]
[out] /public/workspace/licanchengup/link/res_publish/run/cache/disease/SLE_mouse_LN_kidney


# 与健康小鼠肾合并

In [9]:
adata_healty = load_adata(p_cache.joinpath('healty/m_kidney'))
adata_healty

AnnData object with n_obs × n_vars = 43703 × 16272
    obs: '_batch', 'cell_type', 'Cluster_Number', 'pre_cell_type'
    var: 'gene_ids'

In [18]:
adata_merge = sc.concat(
    {'healty':adata_healty,'SLE':adata},
    index_unique=';')

In [29]:
adata_merge.obs['sub_cell_type'] = adata_merge.obs['cell_type']
adata_merge.obs['status'] = adata_merge.obs['_batch']
adata_merge.obs['status'] = adata_merge.obs['status'].mask(
    adata_merge.obs['_batch'].str.startswith('SLE'),
    adata_merge.obs['status'].apply(
    lambda x: '_'.join(
            x.split('_')[
                :-1])
))
adata_merge.obs['status'] = adata_merge.obs['status'].map(lambda k:{
    'lost_7_batches':'healty'
}.setdefault(k,k))

display(adata_merge,
    adata_merge.obs.head(2),adata_merge.obs.shape,
    adata_merge.var.head(2),adata_merge.var.shape,
    adata_merge.obs['status'].value_counts(),
    adata_merge.obs['sub_cell_type'].value_counts()
)

AnnData object with n_obs × n_vars = 69940 × 16261
    obs: '_batch', 'cell_type', 'status', 'sub_cell_type'
    var: 'gene_names', 'gene_ids'

,_batch,cell_type,status,sub_cell_type
AAACCTGAGATATGCA-1;healty,lost_7_batches,LOH,healty,LOH
AAACCTGGTTGTGGCC-1;healty,lost_7_batches,CD-PC,healty,CD-PC


(69940, 4)

,gene_names,gene_ids
Rp1,Rp1,Rp1
Sox17,Sox17,Sox17


(16261, 2)

status
healty     43703
SLE_FMT    16022
SLE        10215
Name: count, dtype: int64

sub_cell_type
LOH         42107
CD-PC        9562
NK           4242
T lymph      3229
CD-Trans     2478
Marco        2056
DCT          1710
Podo         1291
CD-IC         878
Novel2        719
Fib           618
B lymph       567
Neutro        292
Novel1        110
PT             81
Name: count, dtype: int64

In [36]:
display(adata_merge,
    adata_merge.obs.head(2),adata_merge.obs.shape,
    adata_merge.var.head(2),adata_merge.var.shape,
    # adata_merge.obs['_batch'].value_counts()
)
h5ad_to_mtx(adata_merge,p_cache\
            .joinpath('disease','merge_mouse_kideny_healty_SLE')
           )
adata_imm = subset_adata(adata_merge,'cell_type','Macro,Neutro,NK,B lymph,T lymph'.split(','))
display(adata_imm.obs['cell_type'].value_counts())
h5ad_to_mtx(adata_imm,p_cache\
            .joinpath('disease','merge_mouse_kideny_healty_SLE_imm')
           )
# for _ct,_label in zip('NK,T lymph,B lymph,Neutro'.split(','),'NK,T,B,Neu'.split(',')):
for _ct,_label in zip('NK,T lymph,B lymph'.split(','),'NK,T,B'.split(',')):
    _adata = subset_adata(adata_imm,'sub_cell_type',_ct)
    display(_adata.obs['sub_cell_type'].value_counts())
    h5ad_to_mtx(_adata,p_cache\
        .joinpath('disease','merge_mouse_kideny_healty_SLE_imm_{}'.format(_label)))
    del _adata

AnnData object with n_obs × n_vars = 69940 × 16261
    obs: '_batch', 'cell_type', 'status', 'sub_cell_type'
    var: 'gene_names', 'gene_ids'

,_batch,cell_type,status,sub_cell_type
AAACCTGAGATATGCA-1;healty,lost_7_batches,LOH,healty,LOH
AAACCTGGTTGTGGCC-1;healty,lost_7_batches,CD-PC,healty,CD-PC


(69940, 4)

,gene_names,gene_ids
Rp1,Rp1,Rp1
Sox17,Sox17,Sox17


(16261, 2)

...

frist 10 adata.X nonzero elements:
 [[1 1 1 1 3 1 1 8 1 1]]
[out] /public/workspace/licanchengup/link/res_publish/run/cache/disease/merge_mouse_kideny_healty_SLE_imm_B


/public/workspace/licanchengup/link/res_publish/func.py:995: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["gene_names"] = adata.var_names.to_numpy()


sub_cell_type
Neutro    292
Name: count, dtype: int64

frist 10 adata.X nonzero elements:
 [[3 1 1 4 1 1 6 1 1 1]]
[out] /public/workspace/licanchengup/link/res_publish/run/cache/disease/merge_mouse_kideny_healty_SLE_imm_Neu


/public/workspace/licanchengup/link/res_publish/func.py:995: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["gene_names"] = adata.var_names.to_numpy()


# 走个流程看看

In [ ]:
adata_merge
# Saving count data
adata_merge.layers["counts"] = adata_merge.X.copy()
sc.pp.normalize_total(adata_merge)
sc.pp.log1p(adata_merge)
sc.pp.highly_variable_genes(adata_merge, n_top_genes=2000)
sc.tl.pca(adata_merge)
sc.pl.pca_variance_ratio(adata_merge, n_pcs=50, log=True)

In [ ]:
sc.pp.neighbors(adata_merge)
sc.tl.umap(adata_merge)
# Using the igraph implementation and a fixed number of iterations can be
# significantly faster, especially for larger datasets
sc.tl.leiden(adata_merge)
sc.pl.umap(adata_merge, color=["leiden"])

In [ ]:
sc.pl.umap(adata_merge,color=['cell_type'])